In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import concatenate, BatchNormalization
from tensorflow.keras.models import Model, Sequential, load_model, save_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import KFold
from numpy import mean, std
import os
from os import listdir
from numpy import save
from keras.models import load_model
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib.image import imread
from shutil import copyfile
from random import seed, random
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
from tensorflow.keras import backend

In [ ]:
folder = 'Amazon/train-jpg/'

In [ ]:
for i in range(9):
    plt.subplot(330 + 1 + i)
    filename = folder + 'train_' + str(i) + '.jpg'
    image = imread(filename)
    plt.imshow(image)
plt.show()

In [ ]:
#Create Mappings
filename = 'Amazon/train_v2.csv/train_v2.csv'
mapping_csv = read_csv(filename)
#summarize properties
print(mapping_csv.shape)
print(mapping_csv[:10])

In [ ]:
#create set of labels
labels = set()
for i in range(len(mapping_csv)):
    #convert spaced separated tags into an array of tags
    tags = mapping_csv['tags'][i].split(' ')
    #add tags to the set of labels
    labels.update(tags)
    print(tags)

In [ ]:
#convert set to list
labels = list(labels)
#set order
labels.sort()
labels

In [ ]:
#dict that máp labels to integers and reverse
labels_map = {labels[i]:i for i in range(len(labels))}
inv_map = {i:labels[i] for i in range(len(labels))}

In [ ]:
#create mapping of tags to integers given loaded mapping file
def create_tag_mapping(mapping_csv):
    #create set of known tags
    labels = set()
    for i in range(len(mapping_csv)):
        #convert spaced separated tags into array of tags
        tags = mapping_csv['tags'][i].split(' ')
        #add tags to labels set
        labels.update(tags)
    #convert set of labels to list
    labels = list(labels)
    #order set alphabetically
    labels.sort()
    #dict maps labels to integers and reverse
    labels_map = {labels[i]:i for i in range(len(labels))}
    inv_map = {i:labels[i] for i in range(len(labels))}
    return labels_map, inv_map

In [ ]:
filename = 'Amazon/train_v2.csv/train_v2.csv'
mapping_csv = read_csv(filename)
#create a mapping of tags to integers
mapping, inv_mapping = create_tag_mapping(mapping_csv)
print(len(mapping))
print(mapping)

In [ ]:
#create mapping of filename to tags
def create_file_mapping(mapping_csv):
    mapping = dict()
    for i in range(len(mapping_csv)):
        name, tags = mapping_csv['image_name'][i], mapping_csv['tags'][i]
        mapping[name] = tags.split(' ')
    return mapping
mapping = create_file_mapping(mapping_csv)
mapping

Create In-Memory Dataset

In [ ]:
#create one hot encoding for 1 list of tags
def one_hot_encoding(tags, mapping):
    #create empty vector
    encoding = np.zeros(len(mapping), dtype='uint8')
    #mark 1 for each tag in vector
    for tag in tags:
        encoding[mapping[tag]] = 1
    return encoding

In [ ]:
#load all images into memory
def load_dataset(path, file_mapping, tap_mapping):
    photos, targets = list(), list()
    #enumerate files in directory
    for filename in listdir(folder):
        #load image
        photo = load_img(path + filename, target_size=(128,128))
        #convert to numpy array
        photo = img_to_array(photo, dtype='uint8')
        #get tags
        tags = file_mapping[filename[:-4]]
        #one hot encode tags
        target = one_hot_encoding(tags, tap_mapping)
        #store
        photos.append(photo)
        targets.append(target)
    X = np.asarray(photos, dtype='uint8')
    y = np.asarray(targets, dtype='uint8')
    return X, y

In [ ]:
mapping_csv = read_csv(filename)
tag_mapping, _ = create_tag_mapping(mapping_csv)
#create a mapping of filenames to tag lists
file_mapping = create_file_mapping(mapping_csv)
#load jpeg images
X, y = load_dataset(folder, file_mapping, tag_mapping)
print(X.shape, y.shape)
#save both arrays to one file in compressed format
np.savez_compressed('planet_data.npz', X, y)

In [ ]:
#load train and test dataset
def dataset():
    #load dataset
    data = np.load('planet_data.npz')
    X, y = data['arr_0'], data['arr_1']
    #seperate into train and test datasets
    trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.3, random_state=1)
    print(trainX.shape, trainY.shape, testX.shape, testY.shape)
    return trainX, trainY, testX, testY

In [ ]:
trainX, trainY, testX, testY = dataset()
#make all one predictions
train_yhat = np.asarray([np.ones(trainY.shape[1]) for _ in range(trainY.shape[0])])
test_yhat = np.asarray([np.ones(testY.shape[1]) for _ in range(testY.shape[0])])
#evaluate predictions
train_score = fbeta_score(trainY, train_yhat, beta=2, average='samples')
test_score = fbeta_score(testY, test_yhat, beta=2, average='samples')
print(f'All Ones: train={train_score:.3f}, test={test_score:.3f}')

In [ ]:
#calculate fbeta score for multiclass/label classification
def fbeta(y_true, y_pred, beta=2):
    #convert y_true and y_pred to float32
    y_true = backend.cast(y_true, 'float32')
    y_pred = backend.cast(y_pred, 'float32')
    #clip predictions
    y_pred = backend.clip(y_pred, 0, 1)
    #calculate elements
    tp = backend.sum(backend.round(backend.clip(y_true*y_pred, 0, 1)), axis=1)
    fp = backend.sum(backend.round(backend.clip(y_pred-y_true, 0, 1)), axis=1)
    fn = backend.sum(backend.round(backend.clip(y_true-y_pred, 0, 1)), axis=1)
    #calculate precision
    p = tp / (tp + fp + backend.epsilon())
    #calculate recall
    r = tp / (tp + fn + backend.epsilon())
    #calculate fbeta, averaged across each class
    bb = beta**2 #beta squared
    fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
    return fbeta_score

In [ ]:
import keras

In [ ]:
trainX, trainY, testX, testY = dataset()
#make all one predictions
train_yhat = np.asarray([np.ones(trainY.shape[1]) for _ in range(trainY.shape[0])])
test_yhat = np.asarray([np.ones(testY.shape[1]) for _ in range(testY.shape[0])])
#evaluate predictions
train_score = fbeta_score(trainY, train_yhat, beta=2, average='samples')
test_score = fbeta_score(testY, test_yhat, beta=2, average='samples')
print(f'All Ones (sklearn): train={train_score:.3f}, test={test_score:.3f}')
#evaluate predictions with keras
train_score = fbeta(backend.constant(trainY, dtype='float32'), 
                    backend.constant(train_yhat, dtype='float32'))
test_score = fbeta(backend.constant(testY, dtype='float32'),
                   backend.constant(test_yhat, dtype='float32'))
print(f'All Ones (keras): train={train_score:.3f}, test={test_score:.3f}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense, MaxPooling2D, Dropout, Flatten, Layer
from tensorflow.keras.models import Model, Sequential

class Identity(Layer):
    def __init__(self, filters, **kwargs):
        super(Identity, self).__init__(**kwargs)
        filter1, filter2, filter3 = filters
        self.conv1 = Conv2D(filter1, (1,1))
        self.bn1 = BatchNormalization()
        self.act1 = Activation('relu')
        self.conv2 = Conv2D(filter2, (3,3), padding='same')
        self.bn2 = BatchNormalization()
        self.act2 = Activation('relu')
        self.conv3 = Conv2D(filter3, (1,1))
        self.bn3 = BatchNormalization()
    def call(self, input_tensor, training=False):
        #layer1
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.act1(x)
        #layer2
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.act2(x)
        #layer3
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        x = Add()([x, input_tensor])
        x = Activation('relu')(x)
        return x

class Convolutional(Layer):
    def __init__(self, filters, strides=(2,2), **kwargs):
        super(Convolutional, self).__init__(**kwargs)
        filter1, filter2, filter3 = filters
        self.conv1 = Conv2D(filter1, (1,1), strides=strides)
        self.bn1 = BatchNormalization()
        self.act1 = Activation('relu')
        self.conv2 = Conv2D(filter2, (3,3), padding='same')
        self.bn2 = BatchNormalization()
        self.act2 = Activation('relu')
        self.conv3 = Conv2D(filter3, (1,1))
        self.bn3 = BatchNormalization()
        self.shortcut_conv = Conv2D(filter3, (1,1), strides=strides)
        self.shortcut_bn = BatchNormalization()
    def call(self, input_tensor, training=False):
        #layer1
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.act1(x)
        #layer2
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.act2(x)
        #layer3
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        #shortcut layer
        shortcut = self.shortcut_conv(input_tensor)
        shortcut = self.shortcut_bn(shortcut, training=training)
        x = Add()([x, shortcut])
        x = Activation('relu')(x)
        return x

class Resnet(Model):
    def __init__(self, **kwargs):
        super(Resnet, self).__init__(**kwargs)
        #layer 1
        self.conv1 = Conv2D(64, (7, 7), strides=(2, 2), padding='same', input_shape=(128, 128, 3))
        self.bn1 = BatchNormalization()
        self.act1 = Activation('relu')
        self.maxpool1 = MaxPooling2D((3, 3), strides=(2,2), padding='same')
        self.res_block1 = self.resnet_block((64, 64, 256), 3, strides=(1, 1))
        self.res_block2 = self.resnet_block((128, 128, 512), 4, strides=(2, 2))
        self.res_block3 = self.resnet_block((256, 256, 1024), 6, strides=(2, 2))
        self.res_block4 = self.resnet_block((512, 512, 2048), 3, strides=(2, 2))
        self.avgpool = GlobalAveragePooling2D()
        self.flatten = Flatten()
        self.dense1 = Dense(1000, activation='relu')
        self.dropout = Dropout(0.5)
        self.dense2 = Dense(512, activation='relu')
        self.dense3 = Dense(17, activation='sigmoid')
    def resnet_block(self, filters, blocks, strides=(2,2)):
        res_blocks = []
        res_blocks.append(Convolutional(filters, strides))
        for _ in range(1, blocks):
            res_blocks.append(Identity(filters))
        return Sequential(res_blocks)
    def call(self, x):
        #Stage 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.maxpool1(x)
        #Stage 2
        x = self.res_block1(x)
        #stage 3
        x = self.res_block2(x)
        #Stage 4
        x = self.res_block3(x)
        #Stage 5
        x = self.res_block4(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        return self.dense3(x)

In [ ]:
resnet = Resnet()

In [ ]:
def define_model(input_shape=(128, 128, 3), output_shape=17):
    model = Resnet()
    opt = Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', fbeta])
    return model


In [ ]:
#plot diagnostic learning curves
def summarize_diagnostics(histories):
    #plot loss
    plt.subplot(1, 2, 1)
    plt.title('Cross Entropy Loss')
    plt.plot(histories.history['loss'], color='blue', label='train')
    plt.plot(histories.history['val_loss'], color='orange', label='test')
    #plot accuracy
    plt.subplot(1, 2, 2)
    plt.title('Classification Accuracy')
    plt.plot(histories.history['accuracy'], color='blue', label='train')
    plt.plot(histories.history['val_fbeta'], color='orange', label='test')
    plt.show()
    #save plot to file
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    plt.close()

In [ ]:
def run_test_harness():
    train_datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    #prepare iterators
    train_it = train_datagen.flow(trainX, trainY, batch_size=128)
    test_it = test_datagen.flow(testX, testY, batch_size=128)
    #load model
    filename = 'amazon_model.keras'
    model = define_model()
    #fit model
    history = model.fit(train_it, 
                        steps_per_epoch=len(train_it) - 1, 
                        validation_data=test_it, 
                        validation_steps=len(test_it) - 1, 
                        epochs=200, verbose=1)
    #evaluate model
    loss, acc, fbeta = model.evaluate(test_it, 
                            steps=len(test_it), 
                            verbose=0) 
    print(f'loss={loss:.3f}, accuracy={acc:.3f}, fbeta={fbeta:.3f}')
    #learning curves
    summarize_diagnostics(history)
    #save model
    model.save(filename)

In [ ]:
run_test_harness()

In [ ]:
def load_image(filename):
    #load image
    img = load_img(filename, target_size=(128, 128))
    #convert to array
    img = img_to_array(img)
    #reshape into a single sample with 3 channels
    img = img.reshape(1, 128, 128, 3)
    #center pixel data
    img = img.astype('float32')
    img -= [123.68, 116.779, 103.939]
    return img

In [ ]:
#convert predict to tags
def prediction_to_tags(inv_mapping, prediction):
    #round probabilities to 0 or 1
    values = prediction.round()
    #collect all predicted tags
    tags = [inv_mapping[i] for i in range(len(values)) if i in inv_mapping and values[i] == 1.0]
    return tags

In [ ]:
def run_example(inv_mapping):
    folder_path = 'Amazon/test-jpg'
    #load model
    custom_objects = {'Resnet': Resnet,'fbeta': fbeta}
    model = load_model('amazon_model.keras', custom_objects=custom_objects)
    #predict
    #enumerate files in the directory
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if os.path.isfile(file_path):
            img = load_image(file_path)
            result = model.predict(img)
            print(result[0])
            #map prediction to tags
            tags = prediction_to_tags(inv_mapping, result[0])
            print(tags)

In [ ]:
filename = 'Amazon/sample_submission_v2.csv/sample_submission_v2.csv'
mapping_csv = read_csv(filename)
#create mapping of tags to int
_, inv_mapping = create_tag_mapping(mapping_csv)
#entry point, run example
run_example(inv_mapping)